# Deep Learning - Lab Exercise 5


**WARNING:** you must have finished the previous exercise before this one as you will re-use parts of the code.

In the first lab exercise, we built a simple linear classifier.
Although it can give reasonable results on the MNIST dataset (~92.5% of accuracy), deeper neural networks can achieve more the 99% accuracy.
However, it can quickly become really impracical to explicitly code forward and backward passes.
Hence, it is useful to rely on an auto-diff library where we specify the forward pass once, and the backward pass is automatically deduced from the computational graph structure.

In this lab exercise, we will build a small and simple auto-diff lib that mimics the autograd mechanism from Pytorch (of course, we will simplify a lot!)


In [1]:
# import libs that we will use
import os
import numpy as np
import matplotlib.pyplot as plt
import math

# To load the data we will use the script of Gaetan Marceau Caron
# You can download it from the course webiste and move it to the same directory that contains this ipynb file
import dataset_loader

%matplotlib inline

In [2]:
if("mnist.pkl.gz" not in os.listdir(".")):
    # this link doesn't work any more,
    # seach on google for the file "mnist.pkl.gz"
    # and download it
    !wget https://github.com/mnielsen/neural-networks-and-deep-learning/raw/master/data/mnist.pkl.gz


# if you have it somewhere else, you can comment the lines above
# and overwrite the path below
mnist_path = "./mnist.pkl.gz"

In [3]:
# load the 3 splits
train_data, dev_data, test_data = dataset_loader.load_mnist(mnist_path)

## Computation Graph

Instead of directly manipulating numpy arrays, we will manipulate abstraction that contains:
- a value (i.e. a numpy array)
- a bool indicating if we wish to compute the gradient with respect to the value
- the gradient with respect to the value
- the operation to call during backpropagation

There will be two kind of nodes:
- ComputationGraphNode: a generic computation node
- Parameter: a computation node that is used to store parameters of the network. Parameters are always leaf nodes, i.e. they cannot be build from other computation nodes.

Our implementation of the backward pass will be really simple and incorrect in the general case (i.e. won't work with computation graph with loops).
We will just apply the derivative function for a given tensor and then call the ones of its antecedents, recursively.
This simple algorithm is good enough for this exercise.

Note that a real implementation of backprop will store temporary values during forward that can be used during backward to improve computation speed. We do not do that here.


In [4]:
class ComputationGraphNode(object):
    
    def __init__(self, data, require_grad=False):
        # we initialise the value of the node and the grad
        if(not isinstance(data, np.ndarray)):
            data = np.array(data)
        self.value = data
        self.grad = None
        
        self.require_grad = require_grad
        self.func = None
        self.input_nodes = None
        self.func_parameters = []
    
    def set_input_nodes(self, *nodes):
        self.input_nodes = list(nodes)

    def set_func_parameters(self, *func_parameters):
        self.func_parameters = list(func_parameters)
    
    def set_func(self, func):
        self.func = func

    def zero_grad(self):
        if self.grad is not None:
            self.grad.fill(0)

    def set_gradient(self, gradient):
        """
        Accumulate gradient for this tensor
        """
        if gradient.shape != self.value.shape:
            print(gradient.shape, self.value.shape)
            raise RuntimeError("Invalid gradient dimension")
        if self.grad is None:
            self.grad = gradient
        else:
            self.grad += gradient
    
    def backward(self, g=None):
        if g is None:
            g = self.value.copy()
            g.fill(1.)
        self.set_gradient(g)
        if self.func is not None:
            grad_list = self.func.backward(*(self.input_nodes + self.func_parameters + [g]))
            for input_node, ngrad in zip(self.input_nodes, grad_list):
                input_node.backward(ngrad)
    
    def __add__(self, y):
        if not isinstance(y, ComputationGraphNode):
            y = ComputationGraphNode(y)
        return Addition()(self, y)

    def __getitem__(self, slice):
        return Selection()(self, slice)

    def __str__(self):
        return self.value.__str__()

    def __repr__(self):
        return self.value.__str__()

class Parameter(ComputationGraphNode):
    def __init__(self, data, name="default"):
        super().__init__(data, require_grad=True)
        self.name  = name

    def backward(self, g=None):
        if g is not None:
            self.set_gradient(g)

The class `Operation` is a class that three methods you should reimplement only the forward and the backward methods.
* The `forward` method compute the function w.r.t inputs and return a new node that must contains information for backward pass.
* The `backward` functions compute the gradient of the function w.r.t gradient of the output and other informations (forward pass input, parameter of the function...).**It should return a tuple**

For better understanding below two operation are implemented, the selection and the addition (notice that it should not works yet since we do not defined what is a node)

In [5]:
class Operation(object):
    @staticmethod
    def forward(*args):
        raise NotImplementedError("It is an abstract method")
    
    def __call__(self, *args):
        output_node = self.forward(*args)
        output_node.set_func(self)
        return output_node
        
    @staticmethod
    def backward(*args):
        pass
class Addition(Operation):
    @staticmethod
    def forward(x, y):
        output_array = x.value + y.value
        output_node = ComputationGraphNode(output_array)
        output_node.set_input_nodes(x, y)
        return output_node

    @staticmethod
    def backward(x, y, gradient):
        return (gradient, gradient)

class Selection(Operation):
    @staticmethod
    def forward(x, slice):
        np_x = x.value

        output_array = np_x.__getitem__(slice)
        
        output_node = ComputationGraphNode(output_array)
        output_node.set_input_nodes(x)
        output_node.set_func_parameters(slice)

        return output_node
        
    @staticmethod
    def backward(x, slice, gradient):
        np_x = x.value

        cgrad = np_x.copy()
        cgrad.fill(0)
        cgrad.__setitem__(slice, gradient)
        
        return cgrad,

**Question 1** Complete the following class 

In [6]:
class ReLU(Operation):
    @staticmethod
    def forward(x):
        np_x = x.value.copy()
        np_x[np_x < 0] = 0
        output_node = ComputationGraphNode(np_x)
        output_node.set_input_nodes(x)

        return output_node

    @staticmethod
    def backward(x, gradient):
        np_x = x.value
        relu_derivative = (np_x > 0).astype(gradient.dtype)
        grad_input = gradient * relu_derivative

        return (grad_input,)

We recall that :  $$tanh(x)= \frac{e^{z} - e^{-z}}{e^{z} + e^{-z}}$$ 

However we can have stability issues if $||z||$ is large, e.g. $e^{10000}$ will lead to computation error or infinity. Indeed in python using numpy:


>np.exp(10000)


will leads to :

>/tmp/ipykernel_7784/2473798304.py:1: RuntimeWarning: overflow encountered in exp
>np.exp(10000)
>
>inf

We can use the same tricks that the one used in the softmax computation observing the simple following fact: 
$$
\begin{aligned}
 tanh(x) &= \frac{e^{z} - e^{-z}}{e^{z} + e^{-z}} \\
 &= \left(\frac{e^{z} - e^{-z}}{e^{z} + e^{-z}}\right)\frac{e^{-a}}{e^{-a}} \\
 &= \frac{e^{z}e^{-a} - e^{-z}e^{-a}}{e^{z}e^{-a} + e^{-z}e^{-a}} \\
&= \frac{e^{z-a} - e^{-z-a}}{e^{z-a} + e^{-z-a}}
\end{aligned}
$$
Thus we want that $z-a$ or $-z-a$ be small, or in our case lower than $0$.  Thus taking $a$ as the absolute value of $z$ ($|z|$) will leads to have 
$z-a\leq 0$ and $-z-a\leq 0$.


For the backward notice that $tanh'(x) = 1-\sigma(x)^2$


In [7]:
class TanH(Operation):
    @staticmethod
    def TanHCompute(z):
        if not isinstance(z, np.ndarray):
             z = np.array(z)

        abs_z = np.abs(z)
        exp_pos = np.exp(z - abs_z)
        exp_neg = np.exp(-z - abs_z)

        denominator = exp_pos + exp_neg
        epsilon = np.finfo(z.dtype).eps
        tanh_val = (exp_pos - exp_neg) / (denominator + epsilon)

        return tanh_val

    @staticmethod
    def forward(x):
        output_array = TanH.TanHCompute(x.value)

        output_node = ComputationGraphNode(output_array)
        output_node.set_input_nodes(x)

        return output_node

    @staticmethod
    def backward(x, gradient):
        tanh_x_value = TanH.TanHCompute(x.value)
        local_derivative = 1.0 - tanh_x_value**2
        grad_input = gradient * local_derivative

        return (grad_input,)

**Question 2:** Next, we implement the affine transform operation.
You can reuse the code from the third lab exercise, with one major difference: you have to compute the gradient with respect to x too!

In [8]:
class affine_transform(Operation):
    @staticmethod
    def forward(W, b, x):
        np_W = W.value
        np_b = b.value
        np_x = x.value

        if np_x.shape[-1] != np_W.shape[0]:
             raise ValueError(f"Shape mismatch for x @ W: x shape {np_x.shape}, W shape {np_W.shape}")

        try:
            output_array = np_x @ np_W + np_b
        except ValueError as e:
             raise ValueError(f"Error during affine transform x({np_x.shape}) @ W({np_W.shape}) + b({np_b.shape}): {e}")

        output_node = ComputationGraphNode(output_array)
        output_node.set_input_nodes(W, b, x)

        return output_node

    @staticmethod
    def backward(W, b, x, gradient):
        np_W = W.value
        np_x = x.value

        try:
            grad_W = np_x.T @ gradient
        except ValueError as e:
            raise ValueError(f"Shape mismatch calculating grad_W: x.T({np_x.T.shape}) @ grad({gradient.shape}): {e}")

        grad_b = np.sum(gradient, axis=0)
        if b.value.ndim == 2 and b.value.shape[0] == 1:
             if grad_b.ndim == 1:
                 grad_b = grad_b.reshape(1, -1)

        try:
            grad_x = gradient @ np_W.T
        except ValueError as e:
            raise ValueError(f"Shape mismatch calculating grad_x: grad({gradient.shape}) @ W.T({np_W.T.shape}): {e}")

        return grad_W, grad_b, grad_x

**Question 3:** Define the NLL operation

We recall that 
$$nll(x, y)= -log\left(\frac{e^{x_{y}}}{ \sum\limits_{i=1}^n e^{x_{ j}} }\right) = -x_{y} + log(\sum\limits_{i=1}^n e^{x_{ j} })$$

$$
    \begin{align*}
        \frac{\partial nll(x, y)}{\partial x_i} &= - \mathbb{1}_{y = i} + \frac{\partial log(\sum\limits_{i=1}^n e^{x_{ j} })}{\partial\sum\limits_{i=1}^n e^{x_{ j} }}\frac{\sum\limits_{i=1}^n e^{x_{ j} }}{\partial x_i} \\
        &= - \mathbb{1}_{y = i} + \frac{e^{x_i}}{\sum\limits_{i=1}^n e^{x_{ j} }} 
    \end{align*}
$$

In [9]:
class nll(Operation):
    @staticmethod
    def forward(x, y):
        np_x = x.value
        np_y = y.value.flatten().astype(int)

        N = np_x.shape[0]
        if N == 0:
            return ComputationGraphNode(np.array([]))
        if np_y.shape[0] != N:
            raise ValueError(f"Batch size mismatch between x ({N}) and y ({np_y.shape[0]})")

        max_logits = np.max(np_x, axis=1, keepdims=True)
        stable_x = np_x - max_logits
        log_sum_exp = np.log(np.sum(np.exp(stable_x), axis=1, keepdims=True)) + max_logits # Shape (N, 1)

        logits_of_true_class = np_x[np.arange(N), np_y].reshape(N, 1)

        loss_array = -logits_of_true_class + log_sum_exp

        output_node = ComputationGraphNode(loss_array.flatten())
        output_node.set_input_nodes(x, y)

        return output_node

    @staticmethod
    def backward(x, y, gradient):
        np_x = x.value
        np_y = y.value.flatten().astype(int)

        N, C = np_x.shape
        if N == 0:
             return (np.zeros_like(np_x), None)

        max_logits = np.max(np_x, axis=1, keepdims=True)
        stable_x = np_x - max_logits
        exp_x = np.exp(stable_x)
        sum_exp_x = np.sum(exp_x, axis=1, keepdims=True)
        softmax_x = exp_x / sum_exp_x
        indicator = np.zeros_like(np_x)
        indicator[np.arange(N), np_y] = 1.0

        grad_x_local = softmax_x - indicator

        if gradient.ndim == 0:
             grad_x = gradient * grad_x_local
        elif gradient.ndim == 1 and gradient.shape[0] == N:
             grad_x = gradient[:, np.newaxis] * grad_x_local
        else:
             raise ValueError(f"Gradient shape {gradient.shape} incompatible with NLL output ({N},)")

        grad_y = None

        return (grad_x, grad_y)

# Module

Neural networks or parts of neural networks will be stored in Modules.
They implement method to retrieve all parameters of the network and subnetwork.

In [10]:
class Module:
    def __init__(self):
        pass
        
    def parameters(self):
        ret = []
        for name in dir(self):
            o = self.__getattribute__(name)

            if type(o) is Parameter:
                ret.append(o)
            if isinstance(o, Module) or isinstance(o, ModuleList):
                ret.extend(o.parameters())
        return ret

# if you want to store a list of Parameters or Module,
# you must store them in a ModuleList instead of a python list,
# in order to collect the parameters correctly
class ModuleList(list):
    def parameters(self):
        ret = []
        for m in self:
            if type(m) is Parameter:
                ret.append(m)
            elif isinstance(m, Module) or isinstance(m, ModuleList):
                ret.extend(m.parameters())
        return ret

# Initialization and optimization

**Question 1:** Implement the different initialisation methods

In [11]:
def zero_init(b):
    if not np.issubdtype(b.dtype, np.floating):
        b[:] = 0.0
    else:
        b.fill(0)

def glorot_init(W):
    if W.ndim != 2:
        raise ValueError("Glorot initialization expects a 2D weight matrix.")

    fan_in, fan_out = W.shape[0], W.shape[1]
    if fan_in + fan_out == 0:
        W[:, :] = 0.0
        return

    limit = np.sqrt(6.0 / (fan_in + fan_out))
    W[:, :] = np.random.uniform(low=-limit, high=limit, size=W.shape)

def kaiming_init(W):
    if W.ndim != 2:
        raise ValueError("Kaiming initialization expects a 2D weight matrix.")

    fan_in = W.shape[0]
    if fan_in == 0:
        W[:, :] = 0.0
        return

    stddev = np.sqrt(2.0 / fan_in)
    W[:, :] = np.random.normal(loc=0.0, scale=stddev, size=W.shape)

We will implement the Stochastic gradient descent through an object, in the init function this object will store the different parameters (in a list format). The step function will update the parameters (see slides), notice that the gradient is stored in the nodes (grad attribute). Finally it will be necessary after each update to reset all the gradient to zero (in the method zero_grad) because we do not want to accumumlate gradient of all previous step.

**Question 2:** Implement the SGD 

In [12]:
class SGD:
    def __init__(self, params, lr=0.1):
        if not isinstance(params, list):
            raise TypeError("params must be a list of Parameter objects.")
        if lr <= 0.0:
            raise ValueError("Invalid learning rate: {}".format(lr))

        self.params = params
        self.lr = lr

    def step(self):
        for p in self.params:
            if p.require_grad and p.grad is not None:
                if not isinstance(p.value, np.ndarray) or not isinstance(p.grad, np.ndarray):
                     print(f"Warning: Parameter {getattr(p, 'name', 'unnamed')} value or grad is not a numpy array. Skipping update.")
                     continue
                try:
                    p.value -= self.lr * p.grad
                except (TypeError, ValueError) as e:
                     print(f"Warning: Error updating parameter {getattr(p, 'name', 'unnamed')}. "
                           f"Value shape: {p.value.shape}, dtype: {p.value.dtype}. "
                           f"Grad shape: {p.grad.shape}, dtype: {p.grad.dtype}. Error: {e}")

    def zero_grad(self):
        for p in self.params:
            if p.grad is not None:
                if hasattr(p.grad, 'fill'):
                    try:
                        p.grad.fill(0)
                    except Exception as e:
                        print(f"Warning: Could not zero gradient for param {getattr(p, 'name', 'unnamed')} using fill(). Error: {e}. Trying assignment.")
                        try:
                            p.grad = np.zeros_like(p.grad)
                        except Exception as e_assign:
                             print(f"Error: Could not assign zeros to gradient for param {getattr(p, 'name', 'unnamed')}. Error: {e_assign}")

                else:
                    try:
                         p.grad = 0.0 # Or appropriate zero value based on expected type
                    except Exception as e_assign_scalar:
                        print(f"Warning: Gradient for param {getattr(p, 'name', 'unnamed')} is not a numpy array and could not be zeroed. Type: {type(p.grad)}. Error: {e_assign_scalar}")

# Networks and training loop

We first create a simple linear classifier, similar to the first lab exercise.

In [13]:
class LinearNetwork(Module):
    def __init__(self, dim_input, dim_output):
        super().__init__()
        self.dim_input = dim_input
        self.dim_output = dim_output

        W_data = np.empty((dim_input, dim_output), dtype=float)
        b_data = np.empty((1, dim_output), dtype=float)

        self.W = Parameter(W_data, name=f'Linear_{dim_input}x{dim_output}_W')
        self.b = Parameter(b_data, name=f'Linear_{dim_input}x{dim_output}_b')

    def init_parameters(self):
        glorot_init(self.W.value)

        zero_init(self.b.value)

    def forward(self, x):
        if not isinstance(x, ComputationGraphNode):
            x = ComputationGraphNode(x)

        output_node = affine_transform()(self.W, self.b, x)
        return output_node

    def __repr__(self):
        return f"LinearNetwork(dim_input={self.dim_input}, dim_output={self.dim_output})"


In [14]:
np.random.seed(42)

In [15]:
# those lines should be executed correctly
lin1 = LinearNetwork(784, 10)
lin2 = LinearNetwork(10, 5)

lin1.init_parameters()
lin2.init_parameters()

input_image = train_data[0][0]
if input_image.ndim == 1:
    input_image = input_image.reshape(1, -1)

x = ComputationGraphNode(input_image, require_grad=True)

a = lin1.forward(Addition()(x, x))
b = TanH()(a)
c = lin2.forward(b)
c.backward()

print("Gradient of input x (first 10 elements):")
if x.grad is not None:
    print(x.grad.flatten()[:10])
else:
    print("x.grad is None (check require_grad and backward pass)")

Gradient of input x (first 10 elements):
[ 0.08282149  0.32775654 -0.01241754 -0.49801482 -0.07649961 -0.22756026
  0.17287705 -0.15752927  0.23147518 -0.05585501]


We will train several neural networks.
Therefore, we encapsulate the training loop in a function.

**warning**: you have to call optimizer.zero_grad() before each backward pass to reinitialize the gradient of the parameters!

In [16]:
def training_loop(network, optimizer, train_data, dev_data, n_epochs=10):
    X_train, y_train = train_data
    X_dev, y_dev = dev_data
    
    for epoch in range(n_epochs):
        optimizer.zero_grad()
        train_predictions = network.forward(X_train)
        y_train_node = ComputationGraphNode(y_train)
        
        loss_node = nll()(train_predictions, y_train_node)
        mean_loss = np.mean(loss_node.value)
        
        N = len(loss_node.value)
        loss_node.backward(np.ones_like(loss_node.value) / N)
        
        optimizer.step()
        
        dev_predictions = network.forward(X_dev)
        predicted_labels = np.argmax(dev_predictions.value, axis=1)
        accuracy = np.mean(predicted_labels == y_dev)
        
        print(f"mean loss -> {mean_loss} validation accuracy -> {accuracy:.4f}")

In [26]:
dim_input = 28*28
dim_output = 10

network = LinearNetwork(dim_input, dim_output)
network.init_parameters()
optimizer = SGD(network.parameters(), 0.5)

training_loop(network, optimizer, train_data, dev_data, n_epochs=5)

mean loss -> 2.463582859417515 validation accuracy -> 0.3691
mean loss -> 1.9643160077362911 validation accuracy -> 0.6245
mean loss -> 1.6143903074724253 validation accuracy -> 0.7039
mean loss -> 1.3821404731885056 validation accuracy -> 0.7725
mean loss -> 1.2166065665638748 validation accuracy -> 0.7952


After you finished the linear network, you can move to a deep network!

In [18]:
class DeepNetwork(Module):
    def __init__(self, dim_input, dim_output, hidden_dim, n_layers, tanh=False):
        super().__init__()

        self.layers = ModuleList()
        self.use_tanh = tanh

        self.layers.append(LinearNetwork(dim_input, hidden_dim))

        for _ in range(n_layers - 1):
            self.layers.append(LinearNetwork(hidden_dim, hidden_dim))

        self.layers.append(LinearNetwork(hidden_dim, dim_output))

        self.init_parameters()

    def init_parameters(self):
        for layer in self.layers:
            layer.init_parameters()

    def forward(self, x):
        for layer in self.layers[:-1]:
            x = layer.forward(x)
            if self.use_tanh:
                x = TanH()(x)
            else:
                x = ReLU()(x)

        x = self.layers[-1].forward(x)
        return x

In [29]:
dim_input = 28*28
dim_output = 10

network = DeepNetwork(dim_input, dim_output, 100, 2)
network.init_parameters()
optimizer = SGD(network.parameters(), 0.4)

training_loop(network, optimizer, train_data, dev_data, n_epochs=5)

mean loss -> 2.3263948266339063 validation accuracy -> 0.2742
mean loss -> 2.1617729892532047 validation accuracy -> 0.4802
mean loss -> 2.0075487341274383 validation accuracy -> 0.6212
mean loss -> 1.822941641301935 validation accuracy -> 0.6975
mean loss -> 1.6162674656057898 validation accuracy -> 0.7422


## Better Optimizer
Implement the SGD with momentum, notice that you will need to store the cumulated gradient.


In [127]:
class SGDWithMomentum:
    def __init__(self, params, lr=0.1, momentum=0.5):
        self.params = params
        self.lr = lr
        self.momentum = momentum
        self.velocities = []
        for p in params:
            self.velocities.append(np.zeros_like(p.value))

    def step(self):
        for i, p in enumerate(self.params):
            if p.grad is not None:
                self.velocities[i] = self.momentum * self.velocities[i] - self.lr * p.grad
                p.value += self.velocities[i]

    def zero_grad(self):
        for p in self.params:
            if p.grad is not None:
                p.grad.fill(0.0)

## Bonus: Batch SGD
Propose a methods to take into account batch of input